In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,31007
2,Huelva,Confirmados PDIA 14 días,1378
3,Huelva,Tasa PDIA 14 días,"268,5269988502835"
4,Huelva,Confirmados PDIA 7 días,691
5,Huelva,Total Confirmados,31187
6,Huelva,Curados,28649
7,Huelva,Fallecidos,382


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  31007.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8197.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 555 personas en los últimos 7 días 

Un positivo PCR cada 325 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,31007.0,691.0,1378.0,513170.0,134.653234,268.526999,85.0
Huelva-Costa,17862.0,368.0,675.0,289548.0,127.094644,233.121969,48.0
Huelva (capital),8197.0,259.0,443.0,143837.0,180.064935,307.987514,38.0
Condado-Campiña,10073.0,271.0,599.0,156231.0,173.461093,383.406622,31.0
Almonte,1795.0,56.0,134.0,24507.0,228.506141,546.782552,7.0
Sierra de Huelva-Andévalo Central,2726.0,50.0,99.0,67391.0,74.193883,146.903889,6.0
Palos de la Frontera,643.0,22.0,36.0,11742.0,187.361608,306.591722,6.0
Isla Cristina,2147.0,18.0,34.0,21393.0,84.139672,158.930491,4.0
Aljaraque,1077.0,31.0,56.0,21474.0,144.360622,260.780479,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,168.0,5.0,18.0,2211.0,226.142017,814.111262,1.0
Cabezas Rubias,37.0,1.0,5.0,706.0,141.643059,708.215297,0.0
Palma del Condado (La),1142.0,26.0,70.0,10801.0,240.718452,648.088140,1.0
Niebla,232.0,18.0,25.0,4116.0,437.317784,607.385811,1.0
Almonte,1795.0,56.0,134.0,24507.0,228.506141,546.782552,7.0
Moguer,1271.0,51.0,109.0,21867.0,233.228152,498.468011,3.0
Bollullos Par del Condado,971.0,24.0,71.0,14387.0,166.817266,493.501077,1.0
Calañas,189.0,4.0,12.0,2768.0,144.508671,433.526012,0.0
Manzanilla,98.0,3.0,9.0,2118.0,141.643059,424.929178,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cabezas Rubias,37.0,1.0,5.0,706.0,141.643059,708.215297,0.0,0.200000
Villarrasa,168.0,5.0,18.0,2211.0,226.142017,814.111262,1.0,0.277778
San Bartolomé de la Torre,294.0,3.0,10.0,3761.0,79.766020,265.886732,0.0,0.300000
Manzanilla,98.0,3.0,9.0,2118.0,141.643059,424.929178,0.0,0.333333
Rociana del Condado,546.0,4.0,12.0,7939.0,50.384179,151.152538,1.0,0.333333
Villalba del Alcor,481.0,1.0,3.0,3366.0,29.708853,89.126560,0.0,0.333333
Calañas,189.0,4.0,12.0,2768.0,144.508671,433.526012,0.0,0.333333
Bollullos Par del Condado,971.0,24.0,71.0,14387.0,166.817266,493.501077,1.0,0.338028
Cartaya,1328.0,12.0,34.0,20083.0,59.752029,169.297416,2.0,0.352941
